# Time Series Regression Experiments

Placeholder

<img src="./images/TSR.png" width="600" alt="time series regression"> [<i>&#x200B;</i>](./images/TSR.png)

In [ ]:
import numpy as np
from aeon.benchmarking import get_estimator_results, plot_critical_difference
from aeon.datasets import load_regression
from aeon.regression import DummyRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
reg = DummyRegressor()
datasets = ["CardanoSentiment", "Covid3Month", "FloodModeling1"]

# find RMSE for each of our datasets on our estimator
results = {}
for d in datasets:
    train_X, train_y = load_regression(d, split="train")
    test_X, test_y = load_regression(d, split="test")
    reg = reg.fit(train_X, train_y)
    y_pred = reg.predict(test_X)
    results[d] = mean_squared_error(test_y, y_pred, squared=False)

results

In [ ]:
benchmarks = ["InceptionE", "FreshPRINCE", "DrCIF"]
res = get_estimator_results(
    datasets=datasets, estimators=benchmarks, task="regression", type="rmse"
)
res

In [ ]:
res["Dummy"] = results
benchmarks = ["Dummy"] + benchmarks

table = []
for d in datasets:
    row = []
    for b in benchmarks:
        row.append(res[b][d])
    table.append(row)
table = np.array(table)
table

In [ ]:
plt = plot_critical_difference(table, benchmarks)
plt.show()